# Odoo Akko Library

In [1]:
import xmlrpc.client

class Connections:
    def __init__(self, ODOO_URL=None, DB=None, USERNAME=None, PASSWORD=None):
        """
        Bağlantı bilgilerini başlangıçta alabilir veya sonradan connect metodu ile ayarlayabilirsiniz.
        """
        self.ODOO_URL = ODOO_URL
        self.DB = DB
        self.USERNAME = USERNAME
        self.PASSWORD = PASSWORD
        self.uid = None
        self.models = None

        # Eğer tüm bağlantı bilgileri başlangıçta verildiyse, otomatik bağlan
        if all([ODOO_URL, DB, USERNAME, PASSWORD]):
            self.connect()

    def connect(self, ODOO_URL=None, DB=None, USERNAME=None, PASSWORD=None):
        """
        Odoo'ya bağlanır ve kimlik doğrulama yapar.
        """
        # Eğer parametreler verilmişse, sınıf değişkenlerini güncelle
        if ODOO_URL: self.ODOO_URL = ODOO_URL
        if DB: self.DB = DB
        if USERNAME: self.USERNAME = USERNAME
        if PASSWORD: self.PASSWORD = PASSWORD

        # Tüm gerekli parametrelerin var olduğunu kontrol et
        if not all([self.ODOO_URL, self.DB, self.USERNAME, self.PASSWORD]):
            raise ValueError("Bağlantı için tüm parametreler gereklidir: ODOO_URL, DB, USERNAME, PASSWORD")
        
        try:
            # Common API bağlantısını oluşturur
            self.common = xmlrpc.client.ServerProxy(f"{self.ODOO_URL}/xmlrpc/2/common")

            # Kimlik doğrulama
            self.uid = self.common.authenticate(self.DB, self.USERNAME, self.PASSWORD, {})
        
            if not self.uid:
                raise Exception("Odoo kimlik doğrulaması başarısız!")

            # Models API'ye Erişim
            self.models = xmlrpc.client.ServerProxy(f"{self.ODOO_URL}/xmlrpc/2/object")
            
            print(f"Odoo'ya başarıyla bağlanıldı! UID: {self.uid}")
            return self.uid
        
        except Exception as e:
            print(f"Odoo bağlantı hatası: {e}")
            self.common = None
            self.models = None
            self.uid = None
            raise


class OdooModel:
    def __init__(self, connection, model_name):
        """
        Herhangi bir Odoo modeli için CRUD işlemlerini gerçekleştiren sınıf.
        """
        self.connection = connection
        self.model_name = model_name
        
    def _check_connection(self):
        """Bağlantının kurulu olup olmadığını kontrol eder."""
        if not self.connection.uid or not self.connection.models:
            raise Exception("Önce Odoo'ya bağlanmalısınız!")
    
    def create(self, records):
        """
        Modele yeni kayıt(lar) ekler.
        
        Args:
            records: Eklenecek kayıtların listesi. Her kayıt bir dict olmalıdır.
            
        Returns:
            dict: {index_veya_tanımlayıcı: {"id": kayıt_id, "status": durum}}
        """
        self._check_connection()
        
        results = {}
        
        for index, record in enumerate(records):
            try:
                record_id = self.connection.models.execute_kw(
                    self.connection.DB,
                    self.connection.uid,
                    self.connection.PASSWORD,
                    self.model_name,
                    "create",
                    [record]
                )
                
                record_name = record.get('name', str(index))
                results[record_name] = {"id": record_id, "status": "success"}
                
            except Exception as e:
                record_name = record.get('name', str(index))
                results[record_name] = {"status": "error", "message": str(e)}
        
        return results
    
    def search(self, domain=None, fields=None, limit=None, offset=0, order=None):
        """
        Kayıtları arar ve getirir.
        
        Args:
            domain: Arama kriterleri (örn: [['list_price', '>', 100]])
            fields: Getirilecek alanlar (örn: ['id', 'name', 'list_price'])
            limit: Maksimum kayıt sayısı
            offset: Atlanacak kayıt sayısı
            order: Sıralama kriteri (örn: "name ASC")
            
        Returns:
            list: Bulunan kayıtların listesi
        """
        self._check_connection()
        
        domain = domain or []
        fields = fields or []
        
        kwargs = {}
        if fields: kwargs["fields"] = fields
        if limit: kwargs["limit"] = limit
        if offset: kwargs["offset"] = offset
        if order: kwargs["order"] = order
        
        try:
            records = self.connection.models.execute_kw(
                self.connection.DB,
                self.connection.uid,
                self.connection.PASSWORD,
                self.model_name,
                "search_read",
                [domain],
                kwargs
            )
            return records
        except Exception as e:
            print(f"Arama hatası: {e}")
            return []
    
    def update(self, updates):
        """
        Kayıt(lar)ı günceller.
        
        Args:
            updates: Güncellenecek kayıtların listesi.
                    Her öğe {"id": id, "values": {field1: value1, ...}} formatında olmalıdır.
            
        Returns:
            dict: {id: {"status": durum, "message": hata_mesajı}}
        """
        self._check_connection()
        
        results = {}
        
        for update in updates:
            record_id = update.get("id")
            values = update.get("values")
            
            if not record_id or not values:
                continue
                
            try:
                self.connection.models.execute_kw(
                    self.connection.DB,
                    self.connection.uid,
                    self.connection.PASSWORD,
                    self.model_name,
                    "write",
                    [[record_id], values]
                )
                results[record_id] = {"status": "success"}
            except Exception as e:
                results[record_id] = {"status": "error", "message": str(e)}
                
        return results
    
    def delete(self, record_ids):
        """
        Belirtilen ID'lere sahip kayıtları siler.
        
        Args:
            record_ids: Silinecek kayıtların ID listesi
            
        Returns:
            bool: İşlem başarılı ise True
        """
        self._check_connection()
        
        try:
            self.connection.models.execute_kw(
                self.connection.DB,
                self.connection.uid,
                self.connection.PASSWORD,
                self.model_name,
                "unlink",
                [record_ids]
            )
            return True
        except Exception as e:
            print(f"Silme hatası: {e}")
            return False

In [2]:
# Bağlantı oluştur
ODOO_URL = "http://localhost:8069"
DB = "test_db"
USERNAME = "ahmet.nihat.ekici@akko.com.tr"
PASSWORD = "akko123"
conn = Connections(ODOO_URL, DB, USERNAME, PASSWORD)

Odoo'ya başarıyla bağlanıldı! UID: 2


In [3]:
# Ürün modeli için
product_model = OdooModel(conn, "product.pricelist.item")

In [4]:
# Kayıt oluşturma
products = [
    {
    "display_applied_on": "1_product",
    "product_tmpl_id": 5,  # Burada "product" yerine "consu" kullanıyoruz
    "compute_price": "fixed",  # Kategori ID'sini doğru ayarlayın
    "fixed_price": 100.0,  # Satış fiyatı
    "min_quantity": 50.0,  # Maliyet fiyatı
    "pricelist_id": 4
    }
]
results = product_model.create(products)
results

{'0': {'id': 5, 'status': 'success'}}

In [6]:
# Kayıt arama
domain=[['display_applied_on', '=', '1_product']]
fields=['id', 'product_tmpl_id', "pricelist_id"]#name

products = product_model.search(domain, fields)
products

[{'id': 5,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [4, 'Orange (USD)']},
 {'id': 4,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [4, 'Orange (USD)']},
 {'id': 3,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [1, 'Öntanımlı (USD)']},
 {'id': 2,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [1, 'Öntanımlı (USD)']},
 {'id': 1,
  'product_tmpl_id': [25, '[FURN_8220] Four Person Desk'],
  'pricelist_id': [4, 'Orange (USD)']}]

In [7]:
# Kayıt güncelleme
updates = [
    {"id": 3, "values": {'pricelist_id': 4}}
]

results = product_model.update(updates)
results

{3: {'status': 'success'}}

In [8]:
# Kayıt arama
domain=[['display_applied_on', '=', '1_product']]
fields=['id', 'product_tmpl_id', "pricelist_id"]#name

products = product_model.search(domain, fields)
products

[{'id': 5,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [4, 'Orange (USD)']},
 {'id': 4,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [4, 'Orange (USD)']},
 {'id': 3,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [4, 'Orange (USD)']},
 {'id': 2,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [1, 'Öntanımlı (USD)']},
 {'id': 1,
  'product_tmpl_id': [25, '[FURN_8220] Four Person Desk'],
  'pricelist_id': [4, 'Orange (USD)']}]

In [9]:
# Kayıt silme
product_model.delete([3])

True

In [10]:
# Kayıt arama
domain=[['display_applied_on', '=', '1_product']]
fields=['id', 'product_tmpl_id', "pricelist_id"]#name

products = product_model.search(domain, fields)
products

[{'id': 5,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [4, 'Orange (USD)']},
 {'id': 4,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [4, 'Orange (USD)']},
 {'id': 2,
  'product_tmpl_id': [5, '[FURN_7777] Office Chair'],
  'pricelist_id': [1, 'Öntanımlı (USD)']},
 {'id': 1,
  'product_tmpl_id': [25, '[FURN_8220] Four Person Desk'],
  'pricelist_id': [4, 'Orange (USD)']}]